# groupby 공부
pandas로 dplyr처럼 그룹바이에 익숙해지기 위한 공부

링크 : https://rfriend.tistory.com/390?category=675917

# 1. 다양한 groupby 집계방법
- (1) Dictionary를 사용한 groupby
- (2) pd.Series를 사용한 groupby
- (3) functions을 사용한 groupby
- (4) index levels를 사용한 groupby

In [2]:
# importing libraries

import numpy as np
import pandas as pd

# making sample dataset

df = pd.DataFrame(data=np.arange(20).reshape(4, 5),
            columns = ['c1', 'c2', 'c3', 'c4', 'c5'], 
            index = ['r1', 'r2', 'r3', 'r4'])


In [3]:
df

,c1,c2,c3,c4,c5
r1,0,1,2,3,4
r2,5,6,7,8,9
r3,10,11,12,13,14
r4,15,16,17,18,19


## (1) Dicts를 이용한 groupby 예제

### 1-1. 행 기준 Dicts를 이용한 groupby 집계 axis=0

In [4]:
mapping_dict_row = {'r1':'row_g1',
                    'r2':'row_g1',
                    'r3':'row_g2',
                    'r4':'row_g2'}

grouped_by_row = df.groupby(mapping_dict_row)
grouped_by_row.sum()

,c1,c2,c3,c4,c5
row_g1,5,7,9,11,13
row_g2,25,27,29,31,33


### 1-2. 열 기준 Dicts를 이용한 groupby 집계 axis=1

In [5]:
mapping_dict_col = {'c1':'col_g1',
                    'c2':'col_g1',
                    'c3':'col_g2',
                    'c4':'col_g2',
                    'c5':'col_g2'}
grouped_by_col = df.groupby(mapping_dict_col, axis=1)
grouped_by_col.sum()

,col_g1,col_g2
r1,1,9
r2,11,24
r3,21,39
r4,31,54


좀 복잡하게 group을 지정하고 싶을 때 for문 같은걸로 딕셔너리에 그룹을 할당하고 이걸로 그룹바이 하는것도 언젠가 필요할 수도 있을 듯

## (2) Series를 이용한 groupby 집계

### 2-1. 행 기준 Series를 이용한 groupby 집계 axis=0

In [17]:
mapping_series_row = pd.Series(mapping_dict_row)

In [19]:
print(mapping_dict_row)
print(mapping_series_row.index)
print(mapping_series_row.values)

{'r1': 'row_g1', 'r2': 'row_g1', 'r3': 'row_g2', 'r4': 'row_g2'}
Index(['r1', 'r2', 'r3', 'r4'], dtype='object')
['row_g1' 'row_g1' 'row_g2' 'row_g2']


딕셔너리를 pd.Series로 변환하게 되면 key 값이 index로, value값이 values로 들어간다.

In [20]:
df.groupby(mapping_series_row).sum()

,c1,c2,c3,c4,c5
row_g1,5,7,9,11,13
row_g2,25,27,29,31,33


### 2-2. 열 기준 Series를 이용한 groupby 집계 axis=1

In [22]:
mapping_series_col = pd.Series(mapping_dict_col)
df.groupby(mapping_series_col, axis=1).sum()

,col_g1,col_g2
r1,1,9
r2,11,24
r3,21,39
r4,31,54


### 2-3. 열 기준 list를 이용한 groupby 집계 axis=1

In [23]:
mapping_list_col = ['col_g1','col_g1','col_g2','col_g2','col_g2']
df.groupby(mapping_list_col, axis=1).sum()

,col_g1,col_g2
r1,1,9
r2,11,24
r3,21,39
r4,31,54


Dictionary와 달리 series나 list의 경우 group으로 묶어 주려는 행이나 열의 인덱스 개수가 데이터 프레임의 인덱스 개수와 일치해야한다.

## (3) Functions을 이용한 groupby 집계
groupby operator에 사용자 정의 함수를 사용 가능하다. 적용하는 함수 뿐만 아니라 그룹을 정하는 과정 또한 사용자정의 함수가 사용가능하다는 것이다.  
input은 데이터의 원소 하나씩 대입, return은 그 원소가 속할 그룹

In [27]:
print(df)

def row_grp_func(x):
    if x=='r1' or x=='r2':
        row_group = 'row_g1'
    else:
        row_group = 'row_g2'
    return row_group


df.groupby(row_grp_func).sum()

    c1  c2  c3  c4  c5
r1   0   1   2   3   4
r2   5   6   7   8   9
r3  10  11  12  13  14
r4  15  16  17  18  19


,c1,c2,c3,c4,c5
row_g1,5,7,9,11,13
row_g2,25,27,29,31,33


## (4) Index levels를 이용한 groupby 집계

계층적 인덱스(hierarchical index)를 가진 데이터 프레임에 대해서 index levels를 사용하는 방법  
level에 대해서 이름을 부여하여 사용하면 편리하다.  
계층적 인덱스는 R에서는 없는 기능

In [43]:
hier_columns = pd.MultiIndex.from_arrays([['col_g1', 'col_g1', 'col_g2','col_g2', 'col_g2'],
                                        ['c1', 'c2', 'c3', 'c4', 'c5']],
                                        names = ['col_level_1', 'col_level_2'])

hier_df = pd.DataFrame(data = np.arange(20).reshape(4,5),
                    columns = hier_columns,
                    index = ['r1', 'r2', 'r3', 'r4'])
print(hier_df)

hier_df.groupby(level = 'col_level_1',axis=1).sum()

col_level_1 col_g1     col_g2        
col_level_2     c1  c2     c3  c4  c5
r1               0   1      2   3   4
r2               5   6      7   8   9
r3              10  11     12  13  14
r4              15  16     17  18  19


col_level_1,col_g1,col_g2
r1,1,9
r2,11,24
r3,21,39
r4,31,54


# 2. groupby로 그룹별로 반복 작업하기 iteration over groups

pandas의 groupby 객체는 for loop 반복 시에 그룹 이름과 그룹별 데이터셋을 2개의 튜플로 반환한다.  
이러한 특성을 활용하여 그룹별로 for loop 반복작업을 하는데 유용하게 사용할 수 있다.

In [44]:
import numpy as np
import pandas as pd

In [65]:
dat = pd.read_csv('./data/abalone.txt', header=None,
            names = ['sex', 'length', 'diameter', 'height', 
                    'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings'])

dat['length_cat'] = np.where(dat.length > np.median(dat.length), 'length_long', 'length_short')
dat.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings,length_cat
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,length_short
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,length_short
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,length_short
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,length_short
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,length_short


'성별(sex)'로 groupby를 한 후에, for loop를 돌려서 그룹 이름 별로 데이터셋을 프린트해보기

In [70]:
for sex, group_data in dat[['sex', 'length_cat', 'whole_weight', 'rings']].groupby('sex'):
    print(sex, end='')
    print(group_data[:5])

F   sex    length_cat  whole_weight  rings
2    F  length_short        0.6770      9
6    F  length_short        0.7775     20
7    F  length_short        0.7680     16
9    F   length_long        0.8945     19
10   F  length_short        0.6065     14
I   sex    length_cat  whole_weight  rings
4    I  length_short        0.2050      7
5    I  length_short        0.3515      8
16   I  length_short        0.2905      7
21   I  length_short        0.2255     10
42   I  length_short        0.0700      5
M   sex    length_cat  whole_weight  rings
0    M  length_short        0.5140     15
1    M  length_short        0.2255      7
3    M  length_short        0.5160     10
8    M  length_short        0.5095      9
11   M  length_short        0.4060     10


In [73]:
for (sex, length_cat), group_data in dat[['sex', 'length_cat', 'whole_weight', 'rings']].  \
    groupby(['sex', 'length_cat']):
    print(sex,length_cat)
    print(group_data[:2])

F length_long
   sex   length_cat  whole_weight  rings
9    F  length_long        0.8945     19
22   F  length_long        0.9395     12
F length_short
  sex    length_cat  whole_weight  rings
2   F  length_short        0.6770      9
6   F  length_short        0.7775     20
I length_long
    sex   length_cat  whole_weight  rings
509   I  length_long        0.8735     16
510   I  length_long        1.1095     10
I length_short
  sex    length_cat  whole_weight  rings
4   I  length_short        0.2050      7
5   I  length_short        0.3515      8
M length_long
   sex   length_cat  whole_weight  rings
27   M  length_long        0.9310     12
28   M  length_long        0.9365     15
M length_short
  sex    length_cat  whole_weight  rings
0   M  length_short        0.5140     15
1   M  length_short        0.2255      7


In [80]:
list(dat[:10][['sex', 'length_cat', 'whole_weight', 'rings']].groupby('sex'))

[('F',
    sex    length_cat  whole_weight  rings
  2   F  length_short        0.6770      9
  6   F  length_short        0.7775     20
  7   F  length_short        0.7680     16
  9   F   length_long        0.8945     19),
 ('I',
    sex    length_cat  whole_weight  rings
  4   I  length_short        0.2050      7
  5   I  length_short        0.3515      8),
 ('M',
    sex    length_cat  whole_weight  rings
  0   M  length_short        0.5140     15
  1   M  length_short        0.2255      7
  3   M  length_short        0.5160     10
  8   M  length_short        0.5095      9)]

groupby하고나서 list에 넣게 되면 그룹이름과 그룹별 데이터셋을 튜플로 반환한다.  
2개의 튜플형태를 딕셔너리에 넣으면 key와 value로 잘 넣어주는 것을 확인

In [81]:
dict(list(dat[:10][['sex', 'length_cat', 'whole_weight', 'rings']].groupby('sex')))

{'F':   sex    length_cat  whole_weight  rings
 2   F  length_short        0.6770      9
 6   F  length_short        0.7775     20
 7   F  length_short        0.7680     16
 9   F   length_long        0.8945     19,
 'I':   sex    length_cat  whole_weight  rings
 4   I  length_short        0.2050      7
 5   I  length_short        0.3515      8,
 'M':   sex    length_cat  whole_weight  rings
 0   M  length_short        0.5140     15
 1   M  length_short        0.2255      7
 3   M  length_short        0.5160     10
 8   M  length_short        0.5095      9}

In [82]:
dat_sex_group = dict(list(dat[:10][['sex', 'length_cat', 'whole_weight', 'rings']].groupby('sex')))
dat_sex_group['M']

,sex,length_cat,whole_weight,rings
0,M,length_short,0.5140,15
1,M,length_short,0.2255,7
3,M,length_short,0.5160,10
8,M,length_short,0.5095,9
